In [1]:
import pandas as pd
from transformers import AutoTokenizer

In [2]:
df = pd.read_feather('merged.feather')
df.sample(5)

,review_comments,diff_hunk,path,repo,row_id,hunk_id,chunk_id,chunk_text,num_lines
1670829,limit does not return the leftover text,"@@ -21,6 +21,11 @@ const abbreviate = (\n re...",packages/stardog-language-utils/src/namespaceU...,stardog-union/stardog-language-servers,1754556,1.0,1_3-4,+export const splitNamespace = (namespace: str...,2.0
1355674,Configuration should be handled at deploy time...,"@@ -0,0 +1,49 @@\n+import metadataRaw from './...",contracts/scripts/deploy-poc.config.ts,dcgtc/dgrants,1421235,1.0,1_0-39,+import metadataRaw from './grant-round-metada...,40.0
913415,How long does it take to scaffold from a templ...,"@@ -0,0 +1,84 @@\n+// Copyright (c) Microsoft ...",packages/fx-core/src/plugins/resource/cicd/plu...,OfficeDev/TeamsFx,957870,1.0,1_0-69,+// Copyright (c) Microsoft Corporation.\n+// ...,70.0
546078,Why is it named `largArgument`? Was it meant t...,"@@ -0,0 +1,93 @@\n+/*\n+ * SonarQube JavaScrip...",eslint-bridge/src/rules/no-undefined-argument.ts,SonarSource/SonarJS,572608,1.0,1_0-42,+/*\n+ * SonarQube JavaScript Plugin\n+ * Copy...,43.0
1614662,Body medium ne fait pas parti du style guide :...,"@@ -65,6 +64,13 @@ const Body = styled(Colored...",src/ui/theme/typography.ts,pass-culture/pass-culture-app-native,1695822,1.0,1_3-9,+\n+const BodyMedium = styled(ColoredText)({\n...,7.0


In [3]:
context = []

for _, r in df.iterrows():
    chunk = r['chunk_text'] if isinstance(r['chunk_text'], str) else ""
    chunk_lines = chunk.splitlines()
    first = chunk_lines[0].strip() if chunk_lines else ""

    diff = r['diff_hunk'] if isinstance(r['diff_hunk'], str) else ""
    lines = diff.splitlines()

    try:
        i = next(j for j, L in enumerate(lines) if first and first in L)
        s, e = max(i-1, 0), min(i + len(chunk_lines) + 1, len(lines))
        context.append("\n".join(lines[s:e]))
    except StopIteration:
        context.append("")

df['context_text'] = context

In [4]:
df.shape

(1675291, 10)

In [5]:
df.drop(['path', 'repo','num_lines','hunk_id'],axis = 1, inplace = True)
df.shape

(1675291, 6)

In [6]:
df['context_text'][10]

'@@ -0,0 +1,74 @@\n+import { Vue } from "./vue.d";\n+import { VNode, VNodeDirective } from "./vnode.d";\n+\n+type Constructor = {\n+  new (...args: any[]): any;\n+}\n+\n+export interface ComponentOptions {\n+  data?: Object | ( (this: Vue) => Object );\n+  props?: string[] | { [key: string]: PropOptions | Constructor | Constructor[] };\n+  propsData?: Object;\n+  computed?: { [key: string]: ((this: Vue) => any) | ComputedOptions };\n+  methods?: { [key: string]: Function };\n+  watch?: { [key: string]: ({ handler: WatchHandler } & WatchOptions) | WatchHandler | string };\n+\n+  el?: Element | String;\n+  template?: string;\n+  render?(createElement: typeof Vue.prototype.$createElement): VNode;'

In [7]:
def safe_txt(s,n):
    return (s or "")[:n].strip()

print(repr(safe_txt(None,10)))
print(repr(safe_txt('short text', 50)))
print(repr(safe_txt('a'*5000, 10)))    

''
'short text'
'aaaaaaaaaa'


In [10]:
import json

with open("train.jsonl", "w", encoding = "utf-8") as f:
    for _, r in df.iterrows():
        prompt = 'You are a code reviewer. Read the code and provide feedback with concise and helpful review.\n\n'
        
        if isinstance(r.get("context_text"),str) and r.get("context_text").strip():
            prompt += "Context:\n" + safe_txt(r["context_text"],800) + "\n\n"
        
        prompt += "Original(Surrounding lines):\n" + safe_txt(r.get("diff_hunk", ""), 2000) + "\n\n"
        prompt += "Changed code:\n" + safe_txt(r["chunk_text"],4000) +"\n\nReview:"
    
        completion = safe_txt(r.get("review_comments", ""), 1024)
    
        if completion:
            f.write(json.dumps({"prompt" : prompt, "completion" : completion}, ensure_ascii = False) + "\n")
print("train.jsonl created!")

train.jsonl created!


In [11]:
test_prompt = prompt
print(test_prompt[:500])

You are a code reviewer. Read the code and provide feedback with concise and helpful review.

Context:
@@ -0,0 +1,12 @@
+import { Controller, Get } from '@nestjs/common';
+import { AppService } from './app.service';
+
+@Controller()
+export class AppController {

Original(Surrounding lines):
@@ -0,0 +1,12 @@
+import { Controller, Get } from '@nestjs/common';
+import { AppService } from './app.service';
+
+@Controller()
+export class AppController {

Changed code:
+import { Controller, Get } from
